# WAF Auto App - Complete Deployment

This notebook performs an end-to-end deployment:
1. ✅ Deploys the WAF Assessment dashboard to your workspace
2. ✅ Publishes the dashboard and gets the URL
3. ✅ Provides instructions to deploy the app with embedded dashboard

**Prerequisites:**
- Clone this repo to Databricks Repos: `https://github.com/AbhiDatabricks/Databricks-WAF-Light-Tooling`
- Checkout branch: `waf_auto_app`

**What gets deployed:**
- 📊 WAF Assessment Dashboard (Lakeview)
- 🚀 Instructions to deploy the waf-auto app


In [ ]:
# Setup - Get context and credentials
import os, json, requests
from datetime import datetime

# Get user and context info
user = spark.sql("SELECT current_user()").collect()[0][0]
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
api_url = ctx.apiUrl().get()
token = ctx.apiToken().get()

print(f"✅ Connected to: {api_url}")
print(f"✅ User: {user}")


In [ ]:
# Get paths and setup
notebook_path = ctx.notebookPath().get()
base_path = "/".join(notebook_path.split("/")[:-2])  # Go up 2 levels (from waf-auto to repo root)
dashboard_folder = os.path.join(os.getcwd(), "..", "dashboards")
workspace_base_path = f"{base_path}/dashboards"

# Generate timestamp for unique naming
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

print(f"📂 Notebook path: {notebook_path}")
print(f"📂 Dashboard folder: {dashboard_folder}")
print(f"📂 Workspace base: {workspace_base_path}")
print(f"🕐 Timestamp: {timestamp}")
print()


## Step 1: Deploy WAF Assessment Dashboard


In [ ]:
# Deploy WAF Assessment Dashboard
dashboard_filename = "WAF_ASSESSMENTv2.1.lvdash.json"
dashboard_path = os.path.join(dashboard_folder, dashboard_filename)
base_dashboard_name = dashboard_filename.replace(".lvdash.json", "")
dashboard_name = f"{base_dashboard_name}_{timestamp}"

print(f"📊 Deploying: {base_dashboard_name}")
print(f"   Name: {dashboard_name}")
print(f"   File: {dashboard_path}")

# Load dashboard definition
with open(dashboard_path, "r", encoding="utf-8") as f:
    dashboard_def = json.load(f)

# Create or update dashboard
response = requests.post(
    url=f"{api_url}/api/2.0/lakeview/dashboards",
    headers={
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    },
    json={
        "display_name": dashboard_name,
        "parent_path": workspace_base_path,
        "serialized_dashboard": json.dumps(dashboard_def),
        "warehouse_id": None
    }
)

dashboard_id = None
dashboard_url = None

if response.status_code in [200, 201]:
    result = response.json()
    dashboard_id = result.get("dashboard_id", "")
    dashboard_path_result = result.get("path", "")
    dashboard_url = f"{api_url}/sql/dashboardsv3/{dashboard_id}"
    
    print(f"✅ Created dashboard: {dashboard_name}")
    print(f"   📊 Dashboard ID: {dashboard_id}")
    print(f"   🔗 Path: {dashboard_path_result}")
    print(f"   🌐 URL: {dashboard_url}")
elif response.status_code == 400:
    error_json = response.json()
    if error_json.get("error_code") == "RESOURCE_ALREADY_EXISTS":
        print(f"⚠️  Dashboard already exists, finding it...")
        
        # List dashboards
        list_response = requests.get(
            url=f"{api_url}/api/2.0/lakeview/dashboards",
            headers={"Authorization": f"Bearer {token}"}
        )
        
        if list_response.status_code == 200:
            dashboards = list_response.json().get("dashboards", [])
            existing = None
            
            for dash in dashboards:
                if dash.get("display_name") == dashboard_name:
                    existing = dash
                    break
            
            if existing:
                dashboard_id = existing["dashboard_id"]
                dashboard_url = f"{api_url}/sql/dashboardsv3/{dashboard_id}"
                
                # Update it
                update_response = requests.patch(
                    url=f"{api_url}/api/2.0/lakeview/dashboards/{dashboard_id}",
                    headers={
                        "Authorization": f"Bearer {token}",
                        "Content-Type": "application/json"
                    },
                    json={"serialized_dashboard": json.dumps(dashboard_def)}
                )
                
                if update_response.status_code == 200:
                    print(f"✅ Updated dashboard: {dashboard_name}")
                    print(f"   📊 Dashboard ID: {dashboard_id}")
                    print(f"   🌐 URL: {dashboard_url}")
                else:
                    print(f"❌ Failed to update: {update_response.status_code}")
            else:
                print(f"❌ Could not find existing dashboard")
    else:
        print(f"❌ Failed to create dashboard: {response.status_code}")
        print(f"Error: {response.text}")
else:
    print(f"❌ Failed to create dashboard: {response.status_code}")
    print(f"Error: {response.text}")

print()


In [ ]:
# Publish the dashboard
if dashboard_id:
    print(f"📤 Publishing dashboard...")
    publish_response = requests.post(
        url=f"{api_url}/api/2.0/lakeview/dashboards/{dashboard_id}/published",
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json={"embed_credentials": True}
    )
    
    if publish_response.status_code in [200, 201]:
        print(f"✅ Dashboard published!")
        
        # Get org ID for embed URL
        org_id = ctx.tags().get("orgId").get()
        dashboard_direct_url = f"{api_url}/sql/dashboardsv3/{dashboard_id}?o={org_id}"
        dashboard_embed_url = f"{api_url}/embed/dashboardsv3/{dashboard_id}?o={org_id}"
        
        print(f"\n{'='*70}")
        print(f"✅ DASHBOARD DEPLOYED SUCCESSFULLY")
        print(f"{'='*70}")
        print(f"📊 Dashboard Name: {dashboard_name}")
        print(f"🆔 Dashboard ID: {dashboard_id}")
        print(f"🌐 URL: {dashboard_direct_url}")
        print(f"📎 Embed URL: {dashboard_embed_url}")
        print(f"{'='*70}\n")
    else:
        print(f"⚠️  Failed to publish: {publish_response.status_code}")
        print(f"   Error: {publish_response.text}")
        dashboard_embed_url = f"{api_url}/embed/dashboardsv3/{dashboard_id}"
else:
    print(f"❌ No dashboard ID available - deployment failed")
    dashboard_embed_url = None


In [ ]:
## Step 2: Deploy WAF Auto App

Now deploying the Databricks App that will embed this dashboard.


# Deploy WAF Auto App
app_name = "wafauto"
app_url = f"{api_url}/apps/{app_name}"

if dashboard_embed_url:
    print(f"{'='*70}")
    print(f"🚀 DEPLOYING WAF AUTO APP")
    print(f"{'='*70}\n")
    
    # Get repo path for the app
    repo_path = base_path  # This is the repo root
    app_source_path = f"{repo_path}/waf-auto"
    
    print(f"📂 App source path: {app_source_path}")
    print(f"📎 Dashboard embed URL: {dashboard_embed_url}\n")
    
    # Create app using Apps API
    print(f"📱 Creating Databricks App...")
    
    create_payload = {
        "name": app_name,
        "description": "WAF Assessment Dashboard App - Embedded Lakeview Dashboard",
        "source_code_path": app_source_path
    }
    
    # Try to create the app
    create_response = requests.post(
        url=f"{api_url}/api/2.0/apps",
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json=create_payload
    )
    
    if create_response.status_code in [200, 201]:
        print(f"✅ App created: {app_name}")
        app_info = create_response.json()
        
        # Deploy the app
        print(f"🚀 Deploying app...")
        deploy_response = requests.post(
            url=f"{api_url}/api/2.0/apps/{app_name}/deployments",
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            json={
                "source_code_path": app_source_path,
                "mode": "SNAPSHOT"
            }
        )
        
        if deploy_response.status_code in [200, 201]:
            print(f"✅ App deployed successfully!")
            
            # Start the app
            print(f"▶️  Starting app...")
            start_response = requests.post(
                url=f"{api_url}/api/2.0/apps/{app_name}/start",
                headers={
                    "Authorization": f"Bearer {token}",
                    "Content-Type": "application/json"
                },
                json={}
            )
            
            if start_response.status_code in [200, 201]:
                print(f"✅ App started successfully!")
                
                print(f"\n{'='*70}")
                print(f"🎉 DEPLOYMENT COMPLETE!")
                print(f"{'='*70}")
                print(f"📱 App Name: {app_name}")
                print(f"🌐 App URL: {app_url}")
                print(f"📊 Dashboard Embedded: Yes")
                print(f"{'='*70}\n")
                print(f"👉 Click to open app: {app_url}")
            else:
                print(f"⚠️  Failed to start app: {start_response.status_code}")
                print(f"   You can start it manually from: {app_url}")
        else:
            print(f"⚠️  Failed to deploy: {deploy_response.status_code}")
            print(f"   Error: {deploy_response.text}")
    elif create_response.status_code == 409:
        # App already exists
        print(f"ℹ️  App '{app_name}' already exists")
        print(f"   Updating and redeploying...")
        
        # Update and redeploy
        deploy_response = requests.post(
            url=f"{api_url}/api/2.0/apps/{app_name}/deployments",
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            json={
                "source_code_path": app_source_path,
                "mode": "SNAPSHOT"
            }
        )
        
        if deploy_response.status_code in [200, 201]:
            print(f"✅ App redeployed successfully!")
            print(f"\n{'='*70}")
            print(f"🎉 DEPLOYMENT COMPLETE!")
            print(f"{'='*70}")
            print(f"📱 App Name: {app_name}")
            print(f"🌐 App URL: {app_url}")
            print(f"📊 Dashboard Embedded: Yes")
            print(f"{'='*70}\n")
            print(f"👉 Click to open app: {app_url}")
        else:
            print(f"⚠️  Failed to redeploy: {deploy_response.status_code}")
            print(f"   Error: {deploy_response.text}")
    else:
        print(f"❌ Failed to create app: {create_response.status_code}")
        print(f"   Error: {create_response.text}")
        print(f"\n📋 Manual deployment: Go to {api_url}/apps and create app from:")
        print(f"   Repository: https://github.com/AbhiDatabricks/Databricks-WAF-Light-Tooling")
        print(f"   Branch: waf_auto_app")
        print(f"   Path: waf-auto")
        print(f"   Env: DASHBOARD_URL={dashboard_embed_url}")
else:
    print(f"❌ Cannot deploy app - dashboard embed URL not available")


In [ ]:
## 🎉 Deployment Complete!

**✅ Dashboard deployed and published**
**✅ App deployed and started**

Your WAF Assessment Dashboard is now embedded in the Databricks App!

Access the app using the URL shown above.


## Deployment Summary


In [ ]:
# Display final summary with clickable links
displayHTML(f"""
<div style="border: 3px solid #00aa00; padding: 30px; margin: 20px 0; border-radius: 12px; background: linear-gradient(135deg, #f0fff0 0%, #e8f8e8 100%);">
    <h2 style="color: #00aa00; margin-top: 0;">🎉 Deployment Complete!</h2>
    
    <div style="background: white; padding: 20px; margin: 20px 0; border-radius: 8px; border-left: 4px solid #0066cc;">
        <h3 style="margin-top: 0; color: #0066cc;">📊 Dashboard</h3>
        <p><strong>Name:</strong> {DASHBOARD_DISPLAY_NAME}</p>
        <p><strong>ID:</strong> <code>{dashboard_info['id']}</code></p>
        <p><strong>Direct Link:</strong><br>
        <a href="{dashboard_info['url']}" target="_blank" style="color: #0066cc; font-size: 14px;">{dashboard_info['url']}</a></p>
    </div>
    
    <div style="background: white; padding: 20px; margin: 20px 0; border-radius: 8px; border-left: 4px solid #ff6600;">
        <h3 style="margin-top: 0; color: #ff6600;">🚀 WAF Auto App</h3>
        <p><strong>Name:</strong> {app_info['name']}</p>
        <p><strong>Status:</strong> <code>{app_info['status']}</code></p>
        <p><strong>App URL (Dashboard Embedded):</strong><br>
        <a href="{app_info['url']}" target="_blank" style="color: #ff6600; font-size: 16px; font-weight: bold;">{app_info['url']}</a></p>
        <p style="color: #666; font-size: 14px; margin-top: 15px;">👆 Click above to open the app with the embedded dashboard in an iframe</p>
    </div>
    
    <div style="background: #fffef0; padding: 15px; margin: 20px 0; border-radius: 8px; border: 1px solid #ffd700;">
        <h4 style="margin-top: 0; color: #b8860b;">📝 Next Steps</h4>
        <ul style="color: #666;">
            <li>Click the <strong>App URL</strong> above to view your embedded dashboard</li>
            <li>Share the app URL with stakeholders for easy access</li>
            <li>The dashboard is embedded in an iframe with a clean header</li>
            <li>Use <code>/api/health</code> and <code>/api/config</code> endpoints for diagnostics</li>
        </ul>
    </div>
</div>
""")

# Print text version too
print("="*70)
print("DEPLOYMENT SUMMARY")
print("="*70)
print(f"\n📊 Dashboard: {dashboard_info['url']}")
print(f"🚀 App: {app_info['url']}")
print(f"\n✅ All done!")
print()
